In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_cv
from keras import layers
from keras.models import Sequential
from keras.optimizers import Adam

2023-08-29 16:25:00.769084: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 16:25:14.091748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2023-08-29 16:25:24.938591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 16:25:27.149981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 16:25:27.150248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [14]:
train_data = tf.keras.utils.image_dataset_from_directory(
    '../earthshots-data/model-datasets/train', # update this
    validation_split = .15,
    subset='training',
    seed=0,
    image_size = (224, 224),
    batch_size=32,
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '../earthshots-data/model-datasets/train',
    validation_split=.15,
    subset='validation',
    seed=0,
    image_size=(224,224),
    batch_size=32,
)

Found 24051 files belonging to 2 classes.
Using 20444 files for training.
Found 24051 files belonging to 2 classes.
Using 3607 files for validation.


In [15]:
model = tf.keras.applications.ResNet101V2(
    weights=None,
    input_shape=(224,224,3),
    classes=1,
    classifier_activation="sigmoid",
)

# model = tf.keras.models.load_model('models/airfields-effnetV2S-2.keras')


In [16]:
lr = .001

batch_increase = 1

k = batch_increase ** .5

lr = lr * k

optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=lr,
)

acc_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
prec_callback = tf.keras.callbacks.EarlyStopping(monitor='val_precision', patience=5, restore_best_weights=True)

In [17]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()])

In [18]:
history = model.fit(
    x=train_data,
    validation_data = val_data,
    epochs = 100,
    callbacks=[acc_callback]
)

Epoch 1/100


639/639 [==============================] - 165s 186ms/step - loss: 0.2589 - accuracy: 0.9031 - precision_2: 0.7721 - val_loss: 0.6217 - val_accuracy: 0.8303 - val_precision_2: 1.0000
Epoch 2/100
639/639 [==============================] - 114s 178ms/step - loss: 0.1744 - accuracy: 0.9340 - precision_2: 0.8455 - val_loss: 0.7067 - val_accuracy: 0.8356 - val_precision_2: 1.0000
Epoch 3/100
639/639 [==============================] - 113s 177ms/step - loss: 0.1497 - accuracy: 0.9439 - precision_2: 0.8715 - val_loss: 0.7309 - val_accuracy: 0.8298 - val_precision_2: 0.8000
Epoch 4/100
639/639 [==============================] - 113s 177ms/step - loss: 0.1312 - accuracy: 0.9510 - precision_2: 0.8803 - val_loss: 0.2255 - val_accuracy: 0.9263 - val_precision_2: 0.9403
Epoch 5/100
639/639 [==============================] - 114s 177ms/step - loss: 0.1159 - accuracy: 0.9568 - precision_2: 0.8952 - val_loss: 0.3265 - val_accuracy: 0.9149 - val_precision_2: 0.9700
Epoch 6/100
639/639 [================

In [19]:
train_data = tf.keras.utils.image_dataset_from_directory(
    '../earthshots-data/model-datasets/train', # update this
    validation_split = .15,
    subset='training',
    seed=0,
    image_size = (224, 224),
    batch_size=64,
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '../earthshots-data/model-datasets/train',
    validation_split=.15,
    subset='validation',
    seed=0,
    image_size=(224,224),
    batch_size=64,
)

Found 24051 files belonging to 2 classes.
Using 20444 files for training.
Found 24051 files belonging to 2 classes.
Using 3607 files for validation.


In [20]:
batch_increase = 2

k = batch_increase ** .5

lr = lr * k

optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=lr,
)

In [21]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()])

In [22]:
history = model.fit(
    x=train_data,
    validation_data = val_data,
    epochs = 100,
    callbacks=[acc_callback]
)

Epoch 1/100


2023-08-29 18:03:44.342169: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:306] gpu_async_0 cuMemAllocAsync failed to allocate 134217728 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 65011712/12589203456
2023-08-29 18:03:44.342225: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:311] Stats: Limit:                     10408755200
InUse:                      7519483165
MaxInUse:                   7519483165
NumAllocs:                    50370032
MaxAllocSize:               1538326544
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2023-08-29 18:03:44.342286: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:63] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2023-08-29 18:03:44.342294: E tensorflow/compiler/xla/stream_execut

UnknownError: Graph execution error:

Detected at node 'resnet101v2/conv5_block1_2_conv/Conv2D' defined at (most recent call last):
    File "/home/j/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/j/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/j/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/j/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/j/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/j/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home/j/miniconda3/envs/

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

prec = history.history['precision']
val_prec = history.history['val_precision']

epochs_range = range(len(history))

plt.figure(figsize=(8, 8))

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


plt.plot(epochs_range, prec, label='Training Precision')
plt.plot(epochs_range, val_prec, label='Validation Precision')
plt.legend(loc='lower right')
plt.title('Training and Validation Precision')
plt.show()

In [23]:
model.save('models/airfields-resnet101V2-0.keras')